# Trying another approach for other video

Disclaimer: This code was run in another google colab tab and i copied it here to complile the code all at one page.. I want to try another approach without interrupting the code above to prevent error because i need to run the code several time to succeed. Thank you... :)

In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install opencv-python-headless matplotlib
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 25.23 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 51.2 MB/s eta 0:00:00


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
video_path = '/content/drive/MyDrive/6059506-hd_1920_1080_30fps.mp4'

In [13]:
import torch

# Load YOLOv5 model and move to GPU if available
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-2 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [14]:
import cv2

# Load video
video_capture = cv2.VideoCapture(video_path)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Set up output video writer
output_video = cv2.VideoWriter('output_with_detections.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [15]:
import numpy as np
from torchvision.ops import nms
from tqdm.auto import tqdm

# Initialize variables
detection_results = []
frame_index = 0
conf_threshold = 0.25
iou_threshold = 0.6

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to RGB for YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLO inference
    results = model(frame_rgb)
    bbox_data = results.xyxy[0].cpu().numpy()
    boxes, scores, labels = bbox_data[:, :4], bbox_data[:, 4], bbox_data[:, 5]

    # NMS to filter boxes
    boxes_tensor = torch.tensor(boxes)
    scores_tensor = torch.tensor(scores)
    indices = nms(boxes_tensor, scores_tensor, iou_threshold)
    filtered_boxes = bbox_data[indices]

    # Draw detections
    for bbox in filtered_boxes:
        x1, y1, x2, y2, confidence, cls = bbox
        if confidence >= conf_threshold:
            label = model.names[int(cls)]
            color = (0, 255, 0)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Save results
            height, width, _ = frame.shape
            detection_results.append({
                'frame': frame_index,
                'x1': x1 / width,
                'y1': y1 / height,
                'x2': x2 / width,
                'y2': y2 / height,
                'confidence': confidence,
                'class': label
            })

    frame_index += 1
    output_video.write(frame)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

In [16]:
video_capture.release()
output_video.release()

# Convert detection results to DataFrame
import pandas as pd
df_detections = pd.DataFrame(detection_results)
df_detections.to_csv('object_detection_results.csv', index=False)

# Download the output video
from google.colab import files
files.download('output_with_detections.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
df_detections

,frame,x1,y1,x2,y2,confidence,class
0,0,0.447260,0.214482,0.577897,0.772466,0.878978,person
1,0,0.167498,0.321332,0.252075,0.709146,0.845155,person
2,0,0.366306,0.274240,0.480268,0.399319,0.829912,umbrella
3,0,0.700462,0.224800,0.813060,0.385238,0.816562,umbrella
4,0,0.573499,0.301112,0.648582,0.717741,0.782674,person
...,...,...,...,...,...,...,...
17552,820,0.170053,0.462547,0.199565,0.604285,0.441272,person
17553,820,0.445188,0.530515,0.487404,0.637614,0.430763,handbag
17554,820,0.721855,0.534474,0.765825,0.707663,0.326801,suitcase
17555,820,0.348528,0.479473,0.371055,0.550571,0.271467,handbag
